In [ ]:
import tensorflow as tf
from transformers import TFXLMRobertaModel, XLMRobertaTokenizer, TFXLMRobertaForMaskedLM



input_texts = ['query: how much protein should a female eat',
               'query: 南瓜的家常做法',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]

tokenizer = XLMRobertaTokenizer.from_pretrained('intfloat/multilingual-e5-base')
# model = TFXLMRobertaModel.from_pretrained('intfloat/multilingual-e5-base', from_pt=True)

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True)

# outputs = model(**batch_dict)

# print(outputs)


In [ ]:
import numpy as np

wiki_dir = 'wikitext-103-raw/'
PRETRAINING_BATCH_SIZE = 126
PRETRAINING_LEARNING_RATE = 5e-4
PRETRAINING_EPOCHS = 8
VOCABULARY_SIZE = 250002
MASK_RATE = 0.25
PREDICTIONS_PER_SEQ = 32
SEQ_LENGTH = 512



def read_and_filter_lines(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        lines = [line.strip() for line in lines if len(line) > 100]
    return lines

# Paths to the training and validation text files
wiki_train_path = wiki_dir + "wiki.train.raw"
wiki_val_path = wiki_dir + "wiki.valid.raw"

# Create NumPy arrays for training data
wiki_train_lines = np.array(read_and_filter_lines(wiki_train_path))[:1000]

# Create NumPy arrays for validation data
wiki_val_lines = np.array(read_and_filter_lines(wiki_val_path))[:1000]

In [ ]:
wiki_train_lines

In [ ]:
import keras_nlp
from tqdm import tqdm

masker = keras_nlp.layers.MaskedLMMaskGenerator(
    vocabulary_size=VOCABULARY_SIZE,
    mask_selection_rate=MASK_RATE,
    mask_selection_length=PREDICTIONS_PER_SEQ,
    mask_token_id=tokenizer("<mask>")['input_ids'][1]
)


def preprocess(inputs):
    inputs = tokenizer(inputs, max_length=512, padding=True, truncation=True)['input_ids']
    outputs = masker(inputs)
    # Split the masking layer outputs into a (features, labels, and weights)
    # tuple that we can use with keras.Model.fit().
    features = {
        "token_ids": outputs["token_ids"],
        "mask_positions": outputs["mask_positions"],
    }
    labels = outputs["mask_ids"]
    weights = outputs["mask_weights"]
    return features, labels, weights



In [ ]:
pretrain_ds = [preprocess(line) for line in tqdm(wiki_train_lines, desc="Processing Training Data")]
pretrain_val_ds = [preprocess(line) for line in tqdm(wiki_val_lines, desc="Processing Validation Data")]

In [ ]:
features_train, labels_train, weights_train = zip(*pretrain_ds)
features_val, labels_val, weights_val = zip(*pretrain_val_ds)

In [ ]:
encoder_model = TFXLMRobertaForMaskedLM.from_pretrained('intfloat/multilingual-e5-base', from_pt=True)
encoder_model.summary(expand_nested=True)

In [ ]:
# from keras_nlp.backend import keras
# from keras_nlp.backend import ops
import keras

class MaskedLMHead(tf.keras.layers.Layer):

    def __init__(
        self,
        vocabulary_size=None,
        token_embedding=None,
        intermediate_activation="relu",
        activation=None,
        layer_norm_epsilon=1e-05,
        kernel_initializer="glorot_uniform",
        bias_initializer="zeros",
        **kwargs,
    ):
        super().__init__(**kwargs)

        self.vocabulary_size = vocabulary_size
        self.token_embedding = token_embedding
        self.intermediate_activation = keras.activations.get(
            intermediate_activation
        )
        self.activation = keras.activations.get(activation)
        self.layer_norm_epsilon = layer_norm_epsilon
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.bias_initializer = keras.initializers.get(bias_initializer)

        if vocabulary_size is None and token_embedding is None:
            raise ValueError(
                "One of `vocabulary_size` or `token_embedding` must be set. "
                "Received: `vocabulary_size=None`, `token_embedding=None`"
            )

        if token_embedding:
            if vocabulary_size and vocabulary_size != token_embedding.input_dim:
                raise ValueError(
                    "`vocabulary_size` should match the input dimension of the "
                    "of `token_embedding`. Received: "
                    f"`vocabulary_size={vocabulary_size}`, "
                    f"`token_embedding.input_dim={token_embedding.input_dim}`"
                )
            self.vocabulary_size = token_embedding.input_dim

    def build(self, inputs_shape, mask_positions_shape=None):
        if self.token_embedding is not None:
            feature_size = self.token_embedding.output_dim
        else:
            feature_size = inputs_shape[-1]

        self._intermediate_dense = keras.layers.Dense(
            feature_size,
            activation=self.intermediate_activation,
            kernel_initializer=self.kernel_initializer,
            bias_initializer=self.bias_initializer,
            dtype=self.dtype_policy,
            name="intermediate_dense",
        )
        self._intermediate_layer_norm = keras.layers.LayerNormalization(
            epsilon=self.layer_norm_epsilon,
            dtype=self.dtype_policy,
            name="intermediate_layer_norm",
        )
        # The gather length does not affect any of our built variables, so
        # we can pass any value here.
        gather_length = None
        shape = (inputs_shape[0], gather_length, inputs_shape[-1])
        self._intermediate_dense.build(shape)
        shape = (inputs_shape[0], gather_length, feature_size)
        self._intermediate_layer_norm.build(shape)
        if self.token_embedding is None:
            self._kernel = self.add_weight(
                name="output_kernel",
                shape=[feature_size, self.vocabulary_size],
                initializer=self.kernel_initializer,
                dtype=self.dtype,
            )
        self._bias = self.add_weight(
            name="output_bias",
            shape=[self.vocabulary_size],
            initializer=self.bias_initializer,
            dtype=self.dtype,
        )
        self.built = True

    def call(self, inputs, mask_positions):
        # Avoid auto-converting numpy int arrays to float tensors.
        mask_positions = keras.ops.convert_to_tensor(mask_positions, dtype="int")
        # Gather the encoded tokens at the masked indices.
        mask_positions = keras.ops.expand_dims(mask_positions, axis=-1)
        x = keras.ops.take_along_axis(inputs, mask_positions, axis=1)

        # Apply a trainable linear transformation and a layer norm.
        x = self._intermediate_dense(x)
        x = self._intermediate_layer_norm(x)

        # Transform encodings to vocabulary_size predictions.
        if self.token_embedding:
            outputs = self.token_embedding(x, reverse=True)
        else:
            outputs = keras.ops.matmul(x, self._kernel)
        outputs = keras.ops.cast(outputs, self.compute_dtype)
        outputs = outputs + self._bias

        # Apply a final activation.
        if self.activation is not None:
            outputs = self.activation(outputs)

        return outputs

    @classmethod
    def from_config(cls, config):
        embedding = config.get("token_embedding")
        if embedding:
            config["token_embedding"] = keras.layers.deserialize(embedding)
        return super().from_config(config)

    def get_config(self):
        config = super().get_config()
        embedding_config = None
        if self.token_embedding:
            embedding_config = keras.layers.serialize(self.token_embedding)
        config.update(
            {
                "vocabulary_size": self.vocabulary_size,
                "token_embedding": embedding_config,
                "intermediate_activation": keras.activations.serialize(
                    self.intermediate_activation
                ),
                "activation": keras.activations.serialize(self.activation),
                "layer_norm_epsilon": self.layer_norm_epsilon,
                "kernel_initializer": keras.initializers.serialize(
                    self.kernel_initializer
                ),
                "bias_initializer": keras.initializers.serialize(
                    self.bias_initializer
                ),
            }
        )
        return config

    def compute_output_shape(self, inputs_shape, mask_positions_shape):
        return mask_positions_shape + (self.vocabulary_size,)

In [ ]:
# Create the pretraining model by attaching a masked language model head.
inputs = {
    "token_ids": tf.keras.Input(shape=(SEQ_LENGTH,), dtype=tf.int32, name="token_ids"),
    "mask_positions": tf.keras.Input(
        shape=(PREDICTIONS_PER_SEQ,), dtype=tf.int32, name="mask_positions"
    ),
}

encoder_inputs = inputs["token_ids"]

# Encode the tokens.
outputs = encoder_model(encoder_inputs, training=True)


# maskedlm = keras_nlp.layers.MaskedLMHead(vocabulary_size=250002, activation="softmax",)
# outputs = maskedlm(encoded_tokens, mask_positions=inputs["mask_positions"])


pretraining_model = keras.Model(inputs, outputs)
pretraining_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.AdamW(PRETRAINING_LEARNING_RATE),
    weighted_metrics=["sparse_categorical_accuracy"],
    # jit_compile=True,
)



In [ ]:
features_train[0]['token_ids']

In [ ]:
encoder_model.predict([features_train[0]['token_ids']])

In [ ]:
# Pretrain the model on our wiki text dataset.
pretraining_model.fit(
    features_train, labels_train,
    validation_data=(features_val, labels_val),
    epochs=PRETRAINING_EPOCHS,
)

In [ ]:
encoder_model.roberta.embeddings